In [1]:
# Importar librerias
from   bs4 import BeautifulSoup
import requests

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os

import re

# Variables globables
HEADER = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
DEFAULT_UTILS_VERBOSE = True

In [2]:
def get_http_response(url, headers=None, response_type='page', verbose=DEFAULT_UTILS_VERBOSE, debug=False, timeout=10, retry_attempts=3):
    """
    Obtiene la respuesta HTML de una URL.
    Puede aceptar headers personalizados; si no se proporcionan, utiliza unos por defecto.
    La función retorna el resultado HTML como un objeto BeautifulSoup o texto plano.

    Args:
        url (str): La URL de la página web.
        headers (dict, optional): Headers para la solicitud HTTP.
        response_type (str, optional): Tipo de respuesta ('page' para BeautifulSoup, 'text' para texto plano).
        verbose (bool, optional): Si es True, imprime información detallada.
        debug (bool, optional): Si es True, imprime la respuesta HTTP completa.
        timeout (int, optional): Tiempo máximo de espera para la solicitud HTTP en segundos.
        retry_attempts (int, optional): Número de intentos de reintentos en caso de fallo.

    Returns:
        BeautifulSoup object or str: Dependiendo de response_type, retorna un objeto BeautifulSoup o texto plano.

    Raises:
        ValueError: Si response_type no es 'page' o 'text'.
        RuntimeError: Si ocurre un error durante la solicitud HTTP.
    """
    # Validación de parámetros
    if not isinstance(url, str):
        raise ValueError("La URL debe ser una cadena de caracteres.")
    if headers is not None and not isinstance(headers, dict):
        raise ValueError("Headers debe ser un diccionario.")
    if response_type not in ['page', 'text']:
        raise ValueError("response_type debe ser 'page' o 'text'.")
    if not isinstance(verbose, bool):
        raise ValueError("verbose debe ser un valor booleano.")
    if not isinstance(debug, bool):
        raise ValueError("debug debe ser un valor booleano.")
    if not isinstance(timeout, (int, float)):
        raise ValueError("timeout debe ser un número.")
    if not isinstance(retry_attempts, int) or retry_attempts < 0:
        raise ValueError("El número de intentos de reintentos debe ser un entero no negativo.")

    # Definimos los headers por defecto
    if headers is None:
        headers = {
            'user-agent': HEADER
        }

    attempts = 0
    while attempts <= retry_attempts:
        try:
            # Realizamos una solicitud a la página web con timeout
            response = requests.get(url, headers=headers, timeout=timeout)

            # Debug: Imprimimos la respuesta completa si debug es True
            if debug:
                print(f'HTTP response: {response}')

            # Analizamos el contenido HTML de la página web utilizando BeautifulSoup
            page = BeautifulSoup(response.content, 'html.parser')

            # Verbose: Imprimimos información detallada si verbose es True
            if verbose:
                msg  = f'URL [{url}], '
                msg += f'HTTP status [{response.ok}], '
                msg += f'HTTP code [{response.status_code}]'
                print(msg)

            # Validación de la respuesta
            if response.ok:
                if response_type == 'text':
                    return response.text
                else:
                    return page
            else:
                if verbose:
                    msg  = f'URL [{url}], '
                    msg += f'HTTP status [{response.ok}], '
                    msg += f'HTTP code [{response.status_code}], '
                    msg += f'Message [ERROR! Ocurrió un error inesperado al cargar la URL seleccionada]'
                    print(msg)
                return None

        except requests.RequestException as e:
            if verbose:
                logger.error(f'ERROR! Ocurrió un error al realizar la solicitud HTTP para la URL [{url}]. Error: [{e}]')

            # Incrementamos el contador de intentos y esperamos antes de reintentar
            attempts += 1
            if attempts <= retry_attempts:
                time.sleep(1)  # Esperamos 1 segundo antes de realizar el siguiente intento

    # Si llegamos aquí, significa que todos los intentos de reintentos fallaron
    print(f"No se pudo obtener la respuesta HTTP para la URL [{url}] después de {retry_attempts} intentos.")
    return None

In [16]:
# Defino la URL
url = 'https://www.mercadolibre.com.ar/termo-lumilagro-luminox-negro-de-acero-inoxidable-1-l/p/MLA19772281#searchVariation=MLA19772281&position=1&search_layout=stack&type=product&tracking_id=1fb734f3-dd29-43bc-8803-b49ce37013a5'
url

'https://www.mercadolibre.com.ar/termo-lumilagro-luminox-negro-de-acero-inoxidable-1-l/p/MLA19772281#searchVariation=MLA19772281&position=1&search_layout=stack&type=product&tracking_id=1fb734f3-dd29-43bc-8803-b49ce37013a5'

In [17]:
# Pido el contenido HTML
response = get_http_response(url)

URL [https://www.mercadolibre.com.ar/termo-lumilagro-luminox-negro-de-acero-inoxidable-1-l/p/MLA19772281#searchVariation=MLA19772281&position=1&search_layout=stack&type=product&tracking_id=1fb734f3-dd29-43bc-8803-b49ce37013a5], HTTP status [True], HTTP code [200]


In [18]:
rate_box = response.find('span', class_=[
    'ui-search-reviews__ratings',
    'ui-pdp-review__ratings'
])
full_stars = len(rate_box.find_all('svg', class_=[
    'ui-search-icon--star-full',
    'ui-pdp-icon--star-full'
]))
half_stars = len(rate_box.find_all('svg', class_=[
    'ui-search-icon--star-half',
    'ui-pdp-icon--star-half'
]))
rating = full_stars + half_stars * 0.5

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
full_stars

In [ ]:
half_stars

In [ ]:
rating = full_stars + half_stars * 0.5
rating

In [19]:
response


<!DOCTYPE html>

<html lang="es-AR">
<head><link href="https://www.google-analytics.com" rel="preconnect"/><link href="https://www.google.com" rel="preconnect"/><link href="https://data.mercadolibre.com" rel="preconnect"/><link href="https://http2.mlstatic.com" rel="preconnect"/><link href="https://stats.g.doubleclick.net" rel="preconnect"/><link href="https://analytics.mercadolibre.com.ar" rel="preconnect"/><link href="https://analytics.mercadolibre.com" rel="preconnect"/><link href="https://www.google.com.ar" rel="preconnect"/><script nonce="jzK/AerpRK0PqBP4jmD/cA==" type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"agent":"","beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"NRBR-766f4fb616d3a2368ce","applicationID":"1588198355","agentToken":null,"applicationTime":409.767091,"transactionName":"bgQDMEcFXkJZBkYNWldOJBxFFlVCSw9BS3J8NU5LRUsKQUoKVhFWTSgFTG4JTHxlPlNJT3hMOzlOVk1qTRlnOQpiUUxdaE8Z","queueTime":0,"ttGuid":"1f1ee6f970dc352c"}; (window.NREUM|